# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import Callable, Iterable, List, Sequence, Tuple, Optional, Dict

from abc import abstractmethod
import sys
import math
import dataclasses
from collections import namedtuple

import numpy as np
import numpy.typing as npt

import scipy.stats
import scipy.special

# Types

CovMatT = np.ndarray  # Covariance matrix

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (int) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])

Command = namedtuple("Command", ["delta_p", "delta_theta"])


@dataclasses.dataclass(frozen=True)
class Samples:
    """A batch of samples."""
    # These fields are all arrays of shape (N, ),
    # where N is the number of samples
    x: np.ndarray
    y: np.ndarray
    theta: np.ndarray

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass(frozen=True)
class SLAMSamples(Samples):
    """A batch of samples for SLAM."""
    landmarks_id: List[np.ndarray]  # List of length (N, )
    landmarks_x: List[np.ndarray]
    landmarks_y: List[np.ndarray]

    def __post_init__(self):
        fields = [self.landmark_ids, self.landmark_x, self.landmark_y]
        assert len({tuple(a.shape for a in f) for f in fields}) == 1
        return super().__post_init__()


class InferenceAlgo:
    """Interface for an inference algorithm (localization or SLAM)."""

    @abstractmethod
    def init(self, init_mean: np.ndarray, init_cov: CovMatT) -> Pose:
        pass

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> Pose:
        pass


@dataclasses.dataclass(frozen=False)
class Simulator:
    # Initial simulator state
    init_mean: Pose = np.zeros(3)
    # 2x2 covariance matrix of the noise of the initial state
    init_covariance: CovMatT = np.array(
        [[.1, 0, 0], [0, .1, 0], [0, 0, np.deg2rad(5)**2]], dtype=np.float64)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: CovMatT = np.eye(2, dtype=np.float64)
    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: CovMatT = np.array(
        [[1, 0], [0, np.deg2rad(5)**2]], dtype=np.float64)

    # A set of landmarks
    landmarks: Sequence[Landmark] = (
        Landmark(0, -1, -1),
        Landmark(1, 11, -1),
        Landmark(2, 11, 11),
        Landmark(3, -1, 11),
    )

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    def init(self) -> None:
        # Initialize robot state
        self.state = Pose(*np.random.multivariate_normal(
            self.init_mean, self.init_covariance))

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the 
          distance of the movement, and delta_theta(float), the rotation of 
          the movement.
        """
        noise = np.random.multivariate_normal(np.zeros(2),
                                              self.motion_noise_covariance)
        x = self.state.x + math.cos(
            self.state.theta) * (command.delta_p + noise[0])
        y = self.state.y + math.sin(
            self.state.theta) * (command.delta_p + noise[0])
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> List[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        for landmark in self.landmarks:
            r = math.hypot(landmark.x - self.state.x, landmark.x - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta

            noise = np.random.multivariate_normal(np.zeros(2),
                                                  self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[InferenceAlgo] = None
           ) -> Tuple[List[Pose], List[Pose]]:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.

        Returns:
            The true poses of the simulation;
            If inference algorithm is provided, also the estimated poses by the
            inference algorithm.
        """
        true_poses, est_poses = [], []

        self.init()

        true_poses.append(self.state)
        if infer:
            est_poses.append(infer.init(self.init_mean, self.init_covariance))

        for command in commands:
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurement = self.simulate_sensing()
                est_poses.append(infer.update(command, measurement))

        if infer:
            return true_poses, est_poses
        return true_poses


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation of a class method.

    This utility allows one to define methods outside of a class defintion.
    One can therefore provide implementations of a class "incrementally". 
    It is useful for gradually submitting class implementations to catsoop.

    Example:
        >>> class A: 
        ...   foo = NotImplemented

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator





def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def press(event):
    """matplotlib helper function.

    It processes the keyboard event. If the user hits q, it exits the
    program; otherwise, it closes the current plot.
    """
    import matplotlib.pyplot as plt
    if event.key == 'q':
        sys.exit(0)
    plt.close()


def plot_samples(samples: np.ndarray,
                 xmin: float = -1,
                 xmax: float = 1,
                 ymin: float = -1,
                 ymax: float = 1):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
      samples: a numpy array of the samples. This function only plots the first 
        two components: x and y.
      xmin, xmax, ymin, ymax: the range of the x and y axes.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(autoscale_on=False, aspect='equal')
    ax.axis([xmin, xmax, ymin, ymax])
    fig.canvas.mpl_connect('key_press_event', press)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples[:, 0], samples[:, 1], 'g+')
    plt.show()


def plot_trajectories(true_poses: List[Pose],
                      estimated_poses: List[Pose],
                      xmin=-1,
                      xmax=1,
                      ymin=-1,
                      ymax=1):
    """matplotlib helper function. It overlays both the true pose and the
    estimated pose.

    Args:
      true_poses: a numpy array of the true robot trajectory. This function only plots the first two components: x and y.
      estimated_poses: a numpy array of the estimated robot trajectory. This function only plots the first two components: x and y.
      xmin, xmax, ymin, ymax: the range of the x and y axes.
    """

    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.axis([xmin, xmax, ymin, ymax])
    fig.canvas.mpl_connect('key_press_event', press)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plot_traj = lambda poses, color: ax.quiver([p.x for p in poses],
                                               [p.y for p in poses],
                                               np.cos([p.theta for p in poses]),
                                               np.sin([p.theta for p in poses]),
                                               angles="xy",
                                               scale=10,
                                               units="xy",
                                               width=0.01,
                                               color=color)
    plot_traj(true_poses, "blue")
    plot_traj(estimated_poses, "red")
    plt.show()




try:
    from functools import cached_property
except ImportError:
    # Import for Colab (Python==3.7)
    from backports.cached_property import cached_property


@dataclasses.dataclass
class ParticleFilterLocalization(InferenceAlgo):

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark]

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # Number of particles
    num_samples: int = 10

    # The particles
    samples: Samples = None

    def __post_init__(self):
        if self.samples is None:
            zero = np.zeros(self.num_samples)
            self.samples = Samples(zero, zero, zero)
        assert len(self.samples) == self.num_samples

    @cached_property
    def landmarks_id_map(self) -> Dict[int, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(
            *map(np.mean, [self.samples.x, self.samples.y, self.samples.theta]))

    def init(self, init_mean: np.ndarray, init_cov: CovMatT) -> Pose:
        samples = np.random.multivariate_normal(init_mean,
                                                init_cov,
                                                size=self.num_samples)
        self.samples = Samples(samples[:, 0], samples[:, 1], samples[:, 2])
        return self.estimated_pose()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> Pose:
        self.samples = self.update_samples(command, measurements)
        return self.estimated_pose()

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> Samples:
        NotImplemented

    def compute_weights(self, measurements: List[Measurement]) -> np.ndarray:
        NotImplemented

    def update_samples(self, command: Command,
                       measurements: List[Measurement]) -> Samples:
        NotImplemented





## Warmup 1


### Question
Let's make sure we know how to sample from a set of values
based on their weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray, nr_samples: int):
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating 
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

## Warmup 2


### Question
We are going to be sampling from Gaussian
distributions in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: CovMatT,
             num_samples: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_samples: an integer.

    Returns:
      sample: a numpy vector with length num_samples.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## Experiment 1: Sampling from Multivariate Gaussian


### Question

Please generate a 2D scatter plot of 1000 samples drawn from a 2D Gaussian with `mean = [0, 0]` and
`covariance = [[0.05, 0], [[0, 0.01]]`, using the function you just coded: `warmup_2`.
Consider using the helper function provided: `plot_samples`.

<div class="question question-multiplechoice">
<b>Submission Material 1a:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 1a.
</div>

Let's get some intuition about the geometry. Please generate a second 2D scatter plot of
1000 samples drawn from a 2D Gaussian distribution. You should pick the mean and covariance
so that the figure is shown as a ellipse that has a 45-degree angle with the x-y axes.
Specifically, try to modify the mean and covariance to rotate the figure 1a by 45-degree counter-clockwise.

<div class="question question-multiplechoice">
<b>Submission Material 1b:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 1b.
</div>

## Experiment 2: Motion Noises


### Question

We have provided a simulator class for you, that simulates the noisy motion
and sensing of the robot. Let's begin by exploring what happens if we don't
actually do any inference, and assume we always know where the robot is.

We have provided a method `drive_in_a_square` that simulates the motion of the
robot driving around the perimeter of a 10 x 10 square. If you call this
method, you will get a list of poses representing the ''true'' trajectory of
the robot.

Please run the method twice, once with the simulator.motion_noise_covariance
set to `zeros((2, 2))` and once with the simulator.motion_noise_covariance set
to `[[.001, 0], [0, np.deg2rad(3)**2]]`. You should see a significant difference
between the trajectory without noise, and the trajectory with noise. If on each step we
knew the noise, the noise wouldn't be a problem. We could just correct for it
as we steer the robot, but the problem is we **don't** know what the noise is
-- it's just the random perturbations added by nature. We have to use the
sensors to know where the robot is and correct for the
''drift'' in the robot motion. **That** is why we perform inference.

**Hint**: Take a look at the helper function `plot_trajectories` to generate the trajectory plot.

<!-- eighth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 2:</b> In your submitted pdf, please include a single figure which has both
experiment results overlapped on each other. Name this figure as Figure 2.
</div>

Now we've convinced ourselves of the need for inference, let us implement
the two functions we need for sampling-based inference.


## Sampling Question 1


### Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(ParticleFilterLocalization)
def motion_model(self: ParticleFilterLocalization, command: Command) -> Samples:
    """A motion model that simulates a one-step movement of the robot, and
    returns the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.

    Returns:
        new_samples: a batch of new samples obtained by applying the motion model. 
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem1_test():
    # A simulator with no noise
    sim = Simulator(init_covariance=np.zeros((3, 3)),
                    motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_samples = 100
    pf = ParticleFilterLocalization(
        sim.landmarks,
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        num_samples=num_samples,
    )
    pf.init(sim.init_mean,
            [[.01, 0, 0], [0, .01, 0], [0, 0, np.deg2rad(5)**2]])
    samples = pf.motion_model(Command(1, 0))
    assert len(samples) == num_samples
    pose = sim.state
    est_pose = pf.estimated_pose()

    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=1e-1
    ), "Estimated pose should be quite close to simulation without noise"

sampling_problem1_test()

print('Tests passed.')

## Experiment 3: Update with a Motion Model: 1


### Question

Now that you have implemented these, let's play with the noise
parameters. Let's first use the default motion parameters, of $\sigma^2_p = 1$
and $\sigma^2_\theta = 1$. Let's assume our initial estimate of the robot's
pose is a Gaussian with `mean = [0, 0, 0]` and `covariance = [[.01, 0, 0], [0, .01,
0], [0, 0, np.deg2rad(5)**2]]`. Please sample 1000 samples from this initial estimate and
propagate the samples through the motion model twice, using a control of
$\Delta p = 1m$ and $\Delta \theta = 0$ both times.

Please generate plots of the [x, y] component of the samples after the first and
second motions (two separate plots).
This is pretty noisy motion! You should see the samples much
more spread out than they were at the start.  Why is the distribution long and
skinny after one motion, and then really spread out after the second motion?

<!-- second_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 3:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 3a and 3b.
</div>


## Experiment 4: Update with a Motion Model: 2


### Question

Now let's try reducing the noise parameters. Let's set $\sigma^2_p = .01$ and
$\sigma^2_\theta = \text{np.deg2rad}(.5)^2$. Please generate plots of the [x, y]
component of 1000 samples after the same first and second motions as above. The
particle set should be much more compact than before, and track the motion you
commanded.

<!-- third_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 4:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 4a and 4b.
</div>


## Experiment 5: Update with a Motion Model: 3


### Question

For both of the two experiments above, we're taking pretty big motions,
commanding a metre at a time. A mobile robot would generally send commands and
get measurements at a much higher frequency. Let's say the robot is sending
commands at 10 Hz, and travelling at 1 m/s. Please implement a method that
moves the robot the same 2m forward, but in .1 m increments. Please upload a
plot of the [x, y] component of the samples after 2 m.

In this question, please use the same reduced noise parameters: $\sigma^2_p = .01$ and
$\sigma^2_\theta = \text{np.deg2rad}(.5)^2$.

<!-- fourth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 5:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 5.
</div>


## Experiment 6: Update with a Motion Model: 4


### Question

Even though all we did is changed how often we sent a command to the robot,
the sample set is much more spread out. You might think that the uncertainty
in the robot's position is independent of things like control frequency, but
this plot should show you that they aren't. When we adjust the control
frequency, we also have to adjust the model parameters.

Let's set $\sigma^2_p = .001$. In your submitted pdf, plaese include plots of the 
$[x, y]$ component of 1000 samples after the same first and second motions as 
above. The particle set should be much more compact again, and track the motion 
you commanded.

<!-- fifth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 6:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 6.
</div>


## Experiment 7: Update with a Motion Model: 5


### Question

What happens when our motion model has worse rotation noise than translation
noise? Let's leave $\sigma^2_p = .001$ but set $\sigma^2_\theta =
\text{np.deg2rad}(30)^2$. Please generate plots of the $[x, y]$ component of the samples
after 1m, 2m and 3m of motion made in 1m increments. You should see the samples are
tightly contained, but are in a curved shape. This shape is a result of the
non-linear relationship between orientation and position, and is one of the
reasons we often use sampling for localization, rather than a parametric form
such as a Gaussian.

<!-- sixth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 7:</b> In your submitted pdf, please include the generated figures.
Name them as Figure 7a, 7b, and 7c.
</div>



## Experiment 8: Update with a Motion Model: 6


### Question

We still have the relationship between the noise parameters and the control
frequency. Let's leave $\sigma^2_p = .001$ but set $\sigma^2_\theta =
\text{np.deg2rad}(3)^2$. Please generate a plot of the sample distribution after 3m of
motion, but after 30 motions of $.1m$.

<!-- seventh_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 8:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 8.
</div>


## Sampling Question 2


### Question

Let us now add in a measurement model. Let us assume that
there are four landmarks located at $(x, y)$ locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$, 
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = atan2(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how these values are computed.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that is being
applied. (That would be omniscience!) What we can do is work out how likely
each measurement is.  More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. See [Lecture 9 Page 13](https://sicp-s4.mit.edu/_static/fall22/lectures/lec09/L09-Sampling.pdf#page=13)
for details.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples. We encourage you to implement a version
that works for an arbitrary number of landmarks (e.g., more than 4). Although we will
only test your implementation in settings with 4 landmarks.


For reference, our solution is **25** line(s) of code.

In [ ]:
@implementation_for(ParticleFilterLocalization)
def compute_weights(self: ParticleFilterLocalization,
                    measurements: List[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem2_test(compute_weights, pf: ParticleFilterLocalization,
                           measurements: List[Measurement],
                           result_weights: np.ndarray):

    weights = compute_weights(pf, measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

sampling_problem2_test(compute_weights, ParticleFilterLocalization(landmarks=(Landmark(id=0, x=-1, y=-1), Landmark(id=1, x=11, y=-1), Landmark(id=2, x=11, y=11), Landmark(id=3, x=-1, y=11)), motion_noise_covariance=[[0.01, 0], [0, 0.007615435494667714]], sensor_noise_covariance=array([[1.        , 0.        ],
       [0.        , 0.00761544]]), num_samples=100, samples=Samples(x=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), y=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), theta=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))), [Measurement(landmark_id=0, r=0.5011343443312988, b=-2.502908456716752), Measurement(landmark_id=1, r=14.667377828008597, b=-0.06953110921371726), Measurement(landmark_id=2, r=14.667628928750416, b=0.8671443645571141), Measurement(landmark_id=3, r=2.826541268410538, b=1.4546707469539064)], array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01]))
print('Tests passed.')

## Main Question


### Question

Now let's put everything together. We've given you a method
`test_localization()` that does the following:
  * Initializes a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to `np.diag(1, 1, np.deg2rad(5)**2)`.
  * Drives the robot along the same pattern as in the `drive_in_a_square` method,
but on each step, after the command motion do the following:
    * Simulates measurements.
    * Update the samples based on the measurements.

What we want you to do is to write a method for step 4: `update_samples` that takes in a
set of samples, the most recent command, the most recent measurement, and the
motion and sensor covariances and does the following:
  * use the motion_model method to update the sample based on the command
  * use compute_weights to compute the sample weights from the
measurement,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag(.001, .001, np.deg2rad(1)**2)`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **13** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(ParticleFilterLocalization)
def update_samples(self: ParticleFilterLocalization, command: Command,
                   measurements: List[Measurement]) -> Samples:
    """Update the samples, based on the command and the new measurement.

    Args:
        samples: a list of samples.
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.

    Returns:
        new_samples: a numpy array of updated samples.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem3_test(_, pf: ParticleFilterLocalization,
                           command: Command,
                           measurements: List[Measurement], pose: Pose):
    pf.update(command, measurements)
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

sampling_problem3_test(update_samples, ParticleFilterLocalization(landmarks=(Landmark(id=0, x=-1, y=-1), Landmark(id=1, x=11, y=-1), Landmark(id=2, x=11, y=11), Landmark(id=3, x=-1, y=11)), motion_noise_covariance=array([[0.0001, 0.    ],
       [0.    , 0.0001]]), sensor_noise_covariance=array([[1.00000000e-06, 0.00000000e+00],
       [0.00000000e+00, 9.27917724e-08]]), num_samples=10, samples=Samples(x=array([0.24563375, 0.25829234, 0.18484592, 0.2588042 , 0.29137314,
       0.28193171, 0.22561525, 0.19543401, 0.25682821, 0.19015969]), y=array([0.44630584, 0.39492178, 0.38331396, 0.3824855 , 0.42470407,
       0.43606642, 0.32254222, 0.41104762, 0.44219652, 0.39533317]), theta=array([0.76771492, 0.76980286, 0.80652625, 0.82044545, 0.79237316,
       0.8178342 , 0.83964006, 0.84424693, 0.77647332, 0.82173226]))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id=0, r=1.30746602468062, b=-1.8310843773008654), Measurement(landmark_id=1, r=15.877825127575809, b=-0.14353732569298164), Measurement(landmark_id=2, r=15.878843171303302, b=0.7065861702663279), Measurement(landmark_id=3, r=1.3060382572756597, b=1.5629194057471936)], Pose(x=0.2591904038041234, y=0.4149026251238112, theta=0.7846597403889572))
print('Tests passed.')

## Simulation, Sensing, and Localization


### Question

Let's test this out. Using the same motion noise that we used for the initial
experiments at the beginning of this miniproject that convinced us we needed
inference ($\sigma^2_p = .001, \sigma^2_\theta = \text{np.deg2rad}(3)^2$), and sensor
noise where $\sigma^2_r = 1$ and $\sigma^2_b = \text{np.deg2rad}(5)^2$, please
generate a plot of the vehicle trying to execute the same 10m square
trajectory, but in .1m increments. And also plot the x/y mean of the sample
estimates using measurements simulated after each motion using the
`simulator.simulate_sensing` method we provided.

If everything is working, your estimated and ground truth curves should match
up closely, but not exactly. (You should be very suspicious if they match up
exactly.)

Please take a look at the provided implementation of `test_localization()`.
Use this function to simulate the motion, collect sensory data, and perform localization.
You can use the `plot_trajectories` helpful function to generate the plots.

<!-- ninth_experiment() -->
<div class="question question-multiplechoice">
<b>Submission Material 9:</b> In your submitted pdf, please include the generated figure.
Name this figure as Figure 9.
</div>


In [ ]:


def test_localization(simulator: Simulator,
                      num_samples: int = 10) -> Tuple[List[Pose], List[Pose]]:

    return simulator.run(
        drive_in_a_square_commands(),
        ParticleFilterLocalization(
            simulator.landmarks,
            simulator.motion_noise_covariance,
            simulator.sensor_noise_covariance,
            num_samples=num_samples,
        ),
    )